# SMOTE OverSampler

In [1]:
import pandas as pd
import numpy as np
import json

from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from numpy import linspace

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight
#from sklearn.experimental import enable_halving_search_cv
#from sklearn.model_selection import HalvingRandomSearchCV


In [2]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
#test_run = True
test_run = False

In [4]:
df = pd.read_csv("Temp/df_proc_train_set_xgb.csv")

In [5]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
print([i for i in df.columns if 'logP' in i])

['example_SlogP', 'rdkit_desc_SlogP_VSA1', 'rdkit_desc_SlogP_VSA10', 'rdkit_desc_SlogP_VSA11', 'rdkit_desc_SlogP_VSA12', 'rdkit_desc_SlogP_VSA2', 'rdkit_desc_SlogP_VSA3', 'rdkit_desc_SlogP_VSA4', 'rdkit_desc_SlogP_VSA5', 'rdkit_desc_SlogP_VSA6', 'rdkit_desc_SlogP_VSA7', 'rdkit_desc_SlogP_VSA8']


In [7]:
df['sol_category'].value_counts()

2    65835
1     2835
0     2041
Name: sol_category, dtype: int64

In [8]:
if test_run:
    df, _, __, ___ = train_test_split(df, df['sol_category'], train_size=500, stratify=df['sol_category'])

In [9]:
df['sol_category'].value_counts(normalize=True)

2    0.931043
1    0.040093
0    0.028864
Name: sol_category, dtype: float64

In [10]:
#y = pd.read_excel(xl, 'y', header = None)

In [11]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [12]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

70711


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
y = df.iloc[:, 2]

In [14]:
y

0        0
1        0
2        0
3        0
4        0
        ..
70706    2
70707    2
70708    2
70709    2
70710    2
Name: sol_category, Length: 70711, dtype: int8

In [15]:
y.unique()

array([0, 1, 2], dtype=int8)

In [16]:
# Stricter feature selection
### 1. just use ecfp/rdkit+example
### 2. Hyperparatuning (StratifiedKfold+RandomizedSearchCV on X_train and y_train) (max_iter, solver, penalty,C)

# for dataset over all_dataset
#   hyperparameter tuning()

#### removing naN values

In [17]:
list_nan_col = []

for colname in df.columns:
    if not colname == 'sol_category':
        num_nan = 0
        # counting NaN in column
        num_nan = df[colname].isna().sum()
        if num_nan > 1:
            list_nan_col.append(colname)

X.drop(list_nan_col, axis=1, inplace=True)

#### StandardScaler and splitting

In [18]:
# apply StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
X = scaler.fit_transform(X)

# split dataset
StratifiedKFold(n_splits=5)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1,  shuffle=True, stratify=y) #, test_size=0.2, train_size=0.8)

In [19]:
y_train.value_counts()

2    49376
1     2126
0     1531
Name: sol_category, dtype: int64

In [20]:
X_train.shape

(53033, 4324)

## SMOTE

In [21]:
oversample = {
    2 : 50000,
    1 : 12000,
    0 : 10000
}

In [22]:
'''
samples = {}
samples[0] = 1500
samples[1] = 1500
samples[2] = 1500
'''
rus = SMOTE(sampling_strategy=oversample, random_state=0 )
#rus = SMOTE(sampling_strategy='not majority', random_state=0 )#, replacement=True)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [23]:
X_resampled

array([[-0.03079639, -0.02606301, -0.18671033, ..., -0.24571133,
        -0.20651096, -0.36001517],
       [-0.03079639, -0.02606301, -0.18671033, ..., -0.24571133,
        -0.20651096, -0.36001517],
       [-0.03079639, -0.02606301,  5.35589006, ..., -0.24571133,
        -0.20651096,  2.70672773],
       ...,
       [-0.03079639, -0.02606301, -0.18671033, ..., -0.24571133,
        -0.20651096, -0.36001517],
       [-0.03079639, -0.02606301, -0.18671033, ..., -0.24571133,
        -0.20651096, -0.36001517],
       [-0.03079639, -0.02606301, -0.18671033, ..., -0.24571133,
        -0.20651096, -0.36001517]])

In [24]:
y_resampled

0        2
1        2
2        2
3        0
4        2
        ..
71995    2
71996    2
71997    2
71998    2
71999    2
Name: sol_category, Length: 72000, dtype: int8

In [25]:
y_resampled.value_counts()

2    50000
1    12000
0    10000
Name: sol_category, dtype: int64

In [26]:
'''all_dataset = list()
all_dataset.append(X_resampled)
#all_dataset.append(X_ecfp)
#all_dataset.append(X_desc)
#all_dataset.append(X_example)
#all_dataset.append(X_rdd)'''

'all_dataset = list()\nall_dataset.append(X_resampled)\n#all_dataset.append(X_ecfp)\n#all_dataset.append(X_desc)\n#all_dataset.append(X_example)\n#all_dataset.append(X_rdd)'

In [27]:
#from numpy import linspace

In [28]:
'''# setup parameters for xgboost
param = {}
#param['booster'] = ['gbtree', 'gblinear', 'dart']
#param['objective'] = ['binary:logistic']
#param["eval_metric"] = ["error"]
param['eta'] = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5] # boosting learning rate
param['gamma'] = linspace(0.000000001, 1.0, num=11) # min loss red recuired for further partition on leaf node
param['max_depth'] = np.arange(1, 11, 2) # max tree dept for base learners
param['n_estimators'] = np.arange(50, 550, 50)
param['min_child_weight'] = [1] # min sum of instance weight in a child
param['max_delta_step'] = [0] # max delta step allowed for each tree's weight estimate
param['subsample']= [0.5, 1] # subsample ratio of training instance
param['colsample_bytree'] = [1] # subsample ratio of columns when cunstructing each tree
#param['silent'] = [1]
#param['seed'] = [0] # = random_state ???
param['base_score'] = [0.5] # initial prediction score, global bias
#param['random_state'] = [0] # = seed ???
#param['scale_pos_weight'] = ratio

'''

'# setup parameters for xgboost\nparam = {}\n#param[\'booster\'] = [\'gbtree\', \'gblinear\', \'dart\']\n#param[\'objective\'] = [\'binary:logistic\']\n#param["eval_metric"] = ["error"]\nparam[\'eta\'] = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5] # boosting learning rate\nparam[\'gamma\'] = linspace(0.000000001, 1.0, num=11) # min loss red recuired for further partition on leaf node\nparam[\'max_depth\'] = np.arange(1, 11, 2) # max tree dept for base learners\nparam[\'n_estimators\'] = np.arange(50, 550, 50)\nparam[\'min_child_weight\'] = [1] # min sum of instance weight in a child\nparam[\'max_delta_step\'] = [0] # max delta step allowed for each tree\'s weight estimate\nparam[\'subsample\']= [0.5, 1] # subsample ratio of training instance\nparam[\'colsample_bytree\'] = [1] # subsample ratio of columns when cunstructing each tree\n#param[\'silent\'] = [1]\n#param[\'seed\'] = [0] # = random_state ???\nparam[\'base_score\'] = [0.5] # initial prediction score, global bias\n#param[\'r

In [29]:

'''# data set indicator
i=1

#predictions = list()
qwck_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

print('------------------------------------------------------')

for dataset in all_dataset:
    # apply StandardScaler
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)
    dataset = scaler.fit_transform(dataset)

    # split dataset
    StratifiedKFold(n_splits=5)
    X_train, X_test, y_train, y_test = train_test_split(dataset, y_resampled, random_state=1,  shuffle=True, stratify=y_resampled)#, test_size=0.2, train_size=0.8)

    from sklearn.utils import class_weight
    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced',
        y=y_train
        )

    # method
    model = XGBClassifier(**param, n_jobs=5)
    randm_src = RandomizedSearchCV(estimator=model, param_distributions = param, cv = 5, n_iter = 10, n_jobs=5, scoring=qwck_scorer, verbose=1) #verbose=1
    randm_src.fit(X_train, y_train, sample_weight=classes_weights)

    # data set name
    dataset_name = 'X'
    

    # output
    print(f'results of data set {dataset_name}')
    print('------------------------------------------------------')
    print(" Results from Random Search: " )
    print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
    print("\n The best score across ALL searched params:\n", randm_src.best_score_)
    print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')

'''

'# data set indicator\ni=1\n\n#predictions = list()\nqwck_scorer = make_scorer(cohen_kappa_score, weights=\'quadratic\')\n\nprint(\'------------------------------------------------------\')\n\nfor dataset in all_dataset:\n    # apply StandardScaler\n    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)\n    dataset = scaler.fit_transform(dataset)\n\n    # split dataset\n    StratifiedKFold(n_splits=5)\n    X_train, X_test, y_train, y_test = train_test_split(dataset, y_resampled, random_state=1,  shuffle=True, stratify=y_resampled)#, test_size=0.2, train_size=0.8)\n\n    from sklearn.utils import class_weight\n    classes_weights = class_weight.compute_sample_weight(\n        class_weight=\'balanced\',\n        y=y_train\n        )\n\n    # method\n    model = XGBClassifier(**param, n_jobs=5)\n    randm_src = RandomizedSearchCV(estimator=model, param_distributions = param, cv = 5, n_iter = 10, n_jobs=5, scoring=qwck_scorer, verbose=1) #verbose=1\n    randm_s

In [30]:
#randm_src.best_params_

In [31]:
'''hyperparam = randm_src.best_params_

for key in hyperparam:
    if type(hyperparam[key]) == np.int64:
        hyperparam[key] = int(hyperparam[key])
    elif type(hyperparam[key]) == np.float64:
        hyperparam[key] = float(hyperparam[key])
        '''

'hyperparam = randm_src.best_params_\n\nfor key in hyperparam:\n    if type(hyperparam[key]) == np.int64:\n        hyperparam[key] = int(hyperparam[key])\n    elif type(hyperparam[key]) == np.float64:\n        hyperparam[key] = float(hyperparam[key])\n        '

In [32]:
'''
with open('Temp/-xgb_hyperparam.json', 'w') as fp:
    json.dump(hyperparam, fp)
'''

"\nwith open('Temp/-xgb_hyperparam.json', 'w') as fp:\n    json.dump(hyperparam, fp)\n"

In [33]:

# setup parameters for xgboost
params = {}
#params['booster'] = ['gbtree', 'gblinear', 'dart']
#params['objective'] = ['binary:logistic']
#params["eval_metric"] = ["error"]
params['eta'] = 0.0001 #, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5] # boosting learning rate
params['gamma'] = 0.4 #linspace(0.000000001, 1.0, num=11) # min loss red recuired for further partition on leaf node
params['max_depth'] = 5 #np.arange(1, 11, 2) # max tree dept for base learners
params['n_estimators'] = 250 #np.arange(50, 550, 50)
params['min_child_weight'] = 1 # min sum of instance weight in a child
params['max_delta_step'] = 0 # max delta step allowed for each tree's weight estimate
params['subsample']= 0.5 #[0.5, 1] # subsample ratio of training instance
params['colsample_bytree'] = 1 # subsample ratio of columns when cunstructing each tree
#params['silent'] = [1]
#params['seed'] = [0] # = random_state ???
params['base_score'] = 0.5 # initial prediction score, global bias
#params['random_state'] = [0] # = seed ???
#params['scale_pos_weight'] = ratio
params['n_jobs'] = 5
params['eval_metric'] = 'mlogloss'

In [34]:

classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_resampled
    )

model = XGBClassifier(**params)

model.fit(X_resampled, y_resampled, sample_weight=classes_weights)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.0001,
              eval_metric=None, feature_types=None, gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=9.99999975e-05,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=250,
              n_jobs=5, num_parallel_tree=1, objective='multi:softprob', ...)

In [35]:

valPredictions = model.predict(X_val)

#quad kappa score
sk_quad_kappa = cohen_kappa_score(y_val, valPredictions, weights='quadratic')
    
# printing results
print(f'quadratic kappa score of the validation data set: {sk_quad_kappa}')


quadratic kappa score of the validation data set: 0.003983515313151442


In [36]:


real_test_set = pd.read_csv('Temp/df_proc_test_set.csv')
sub_template = pd.read_csv('Data/submission_template_rdm.csv')
(sub_template['Id'] == real_test_set['Id']).value_counts()


True    30307
Name: Id, dtype: int64

In [37]:

real_test_set.drop(columns=['Id', 'smiles'], inplace=True)

# apply StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(real_test_set)
real_test_set = scaler.fit_transform(real_test_set)

# prediction
realtestPredictions = model.predict(real_test_set)
sub_template['pred'] = realtestPredictions


ValueError: Feature shape mismatch, expected: 4324, got 4271

In [ ]:
set(realtestPredictions)

{0, 1, 2}

In [ ]:

sub_template['pred'].value_counts()


2    28714
1     1030
0      563
Name: pred, dtype: int64

In [ ]:
#sub_template.to_csv('Submissions/submission_24_11_C-men.csv', index=False)

## Score

In [ ]:
#len(testPredictions)
#testPredictions

In [ ]:
#y_test

In [ ]:
#set(predictions[0])

In [ ]:
'''i=1

for preds in predictions:
    # defining name
    if i == 1:
        dataset_name = 'X'
    elif i == 2:
        dataset_name = 'X_exfp'
    elif i == 3:
        dataset_name = 'X_desc'
    
    i += 1

    #quad kappa score
    sk_quad_kappa = cohen_kappa_score(y_test, testPredictions, weights='quadratic')
    
    # printing results
    print(f'quadratic kappa score of data set {dataset_name}: {sk_quad_kappa}')'''

"i=1\n\nfor preds in predictions:\n    # defining name\n    if i == 1:\n        dataset_name = 'X'\n    elif i == 2:\n        dataset_name = 'X_exfp'\n    elif i == 3:\n        dataset_name = 'X_desc'\n    \n    i += 1\n\n    #quad kappa score\n    sk_quad_kappa = cohen_kappa_score(y_test, testPredictions, weights='quadratic')\n    \n    # printing results\n    print(f'quadratic kappa score of data set {dataset_name}: {sk_quad_kappa}')"

In [ ]:
#sk_cohen_kappa = cohen_kappa_score(y_prediction, y_test, weights='quadratic')
#print(f'quadratic cohen kappa: {sk_cohen_kappa}')

In [ ]:
#from sklearn.metrics import confusion_matrix

In [ ]:
#confusion_matrix(y_test, y_prediction)

In [ ]:
#df_y[0].value_counts()

In [ ]:
#len(y_test)